In [53]:
#scrapping data from Trip advisor
import requests             
from bs4 import BeautifulSoup 
import sys
import csv                  
import webbrowser
import io
import re
import pandas as pd

In [70]:

# get urls
urls=[]
i = 0
j = 0
#j = 10
#div_page = no_reviews // 10
div_page = 72 // 10
#print(div_page)
while i < div_page+1:
    page_num = str(j)
    url='https://www.tripadvisor.com/Restaurant_Review-g189506-d3783851-or'+page_num+'-Reviews-Palia_Istoria-Trikala_Trikala_Region_Thessaly.html'
    
    urls.append(url)
    j = j + 10
    i=i+1
    #print(url)


#for k in urls:
#    print(k)


In [71]:
task_no =0
review_data={}
c_page=0

In [72]:
counter_com = 0
for url_i in urls:    
    response= requests.get(url_i)
    data = response.text
    soup = BeautifulSoup(data,'html.parser')
    reviews = soup.find_all('div',{'class':'review-container'})


    #No of Reviewers 
    no_reviews = soup.find('span',{"class":"count"})
    if no_reviews == None:
        no_reviews = 'Undefined Number'

    else:
        no_reviews = no_reviews.text
        no_reviews = int(re.search(r'\d+', no_reviews).group(0))


    for review in reviews:
        ratingdates = review.find('span',{'class':'ratingDate'}).get('title')
        ratings = float(review.find('span',{'class':re.compile(r'ui_bubble_rating bubble_*')}).get("class")[1].split("_")[1])/10
        rating_title = review.find('span',{'class':'noQuotes'}).text
        description = review.find('p',{'class':'partial_entry'}).text
        site= 'Trip Advisor'
        if ratings < 4:
            groundTruth = 'negative'
        else:
            groundTruth = 'positive'
        task_no=task_no + 1
        review_data[task_no] = [ratings, ratingdates, description,rating_title,site, groundTruth]
        #print(task_no, '\nRatings:', ratings, '\nDates:', ratingdates, '\nDescription:', description,  '\nTitle:',  rating_title, '\nSite:', site ,'\nGroundTruth:', groundTruth,'\n---')





In [73]:
task_no

76

In [74]:
review_df = pd.DataFrame.from_dict(review_data, orient = 'index', columns = ['Ratings','Review Date', 'Description', 'Review Title', 'Site', 'GroundTruth' ])

In [75]:
review_df.head(5)

,Ratings,Review Date,Description,Review Title,Site,GroundTruth
1,5.0,"February 26, 2020",Everything was so good and delicious! The owne...,Our perfect lunch in Trikala,Trip Advisor,positive
2,5.0,"January 18, 2020",The food was very good ! The pork knee was ama...,Amazing,Trip Advisor,positive
3,4.0,"April 21, 2019",Enjoyed eating here on my first business trip ...,Honest tasty food at a sweet price,Trip Advisor,positive
4,5.0,"December 1, 2018",We ate two days ago and the food was great! Wa...,Warn place with great food!,Trip Advisor,positive
5,5.0,"July 14, 2018",This is only a couple of blocks from the hotel...,Wonderful eatery on the main restaurant district.,Trip Advisor,positive


In [76]:


excel_path = "C:\\Users\\Petros\\Desktop\\reviews.xlsx"
def append_df_to_excel(review_df, excel_path):
    df_excel = pd.read_excel(excel_path)
    result = pd.concat([df_excel, review_df], ignore_index=True)
    result.to_excel(excel_path, index=False)


append_df_to_excel(review_df, r"C:\\Users\\Petros\\Desktop\\reviews.xlsx")

C:\Users\Petros\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
